# Mount drive and append path to PYTONPATH


In [ ]:
import os
import sys

from google.colab import drive

drive.mount("/content/drive")
sys.path.append("/content/drive/MyDrive/DeepLCMS/train_google_colab")

# Import and install libraries

In [ ]:
%%capture
!pip install lightning
!pip install timm
!pip install torchinfo
!pip install torchmetrics
!pip install torchcam

In [ ]:
from pathlib import Path

import colab_functions
import colab_utils
import pandas as pd
import prepare_data

import timm
import torch
import torchinfo
import train_NN
from lightning.pytorch.loggers import CSVLogger
from timm import create_model
import torchmetrics
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import Callback
from lightning.pytorch import Trainer
from lightning.pytorch import LightningModule
from sklearn import metrics
import numpy as np

In [ ]:
# Set the CUDA_VISIBLE_DEVICES environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Unzip data

In [ ]:
!unzip -q "*.zip"

# Check if GPU is used

In [ ]:
device = colab_functions.get_device()

# Final training with optimized settings

In [ ]:
model = train_NN.Resnet_model()

In [ ]:
train_NN.show_architecture(model)

In [ ]:
(
    preprocess_train,
    preprocess_val,
    preprocess_test,
) = prepare_data.get_timm_transforms(model)

(
    train_dataloader,
    val_dataloader,
    test_dataloader,
) = prepare_data.get_dataloaders(
    preprocess_train=preprocess_train,
    preprocess_val=preprocess_val,
    preprocess_test=preprocess_test,
)

In [ ]:
# initialize checkpoints
early_stop = EarlyStopping(monitor="val_loss", patience=10, mode="min")
checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor="val_loss", mode="min")

logger = CSVLogger("logs", name="final_training")

trainer = Trainer(
    max_epochs=50,
    callbacks=[early_stop, checkpoint_callback],
    log_every_n_steps=1,
    logger=logger,
    enable_checkpointing=True,
)

trainer.fit(
    model=model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

In [ ]:
colab_functions.plot_experiment_results(colab_functions.get_experiment_results())

In [ ]:
print(checkpoint_callback.best_model_path)  # prints path to the best model's checkpoint
print(checkpoint_callback.best_model_score)  #

In [ ]:
print(checkpoint_callback.best_model_path)  # prints path to the best model's checkpoint
print(checkpoint_callback.best_model_score)  # and prints it score

best_model = train_NN.Resnet_model.load_from_checkpoint(
    checkpoint_callback.best_model_path
)

# Evaluate the test set


In [ ]:
predictions = trainer.predict(best_model, test_dataloader)

As you can see our model performs exceptionally good:
Accuracy: 0.90 | F1: 0.93 | Precision: 0.86 | Recall: 1.00


In [ ]:
colab_functions.evaluate_predictions(
    logits=predictions, test_dataloader=test_dataloader
)

# Random predictions

As demonstrated below, generating random predictions aligned with the distribution of our test set results in:

*   Accuracy of 0.50
*   F1 score of 0.59
*   Precision of 0.73
* Recall of 0.50

indicating that our model significantly outperforms random guessing.


In [ ]:
from sklearn import metrics
import numpy as np

RANGE = 1000

results = {"Accuracy": [], "Precision": [], "Recall": [], "F1": []}

for _ in range(RANGE):
    true = np.concatenate([np.zeros(8), np.ones(22)])
    predicted = np.random.choice([0, 1], 30)

    scores = {
        "Accuracy": metrics.accuracy_score(true, predicted),
        "Precision": metrics.precision_score(true, predicted),
        "Recall": metrics.recall_score(true, predicted),
        "F1": metrics.f1_score(true, predicted),
    }

    for key, value in scores.items():
        results[key].append(value)

mean_results = {key: np.mean(values) for key, values in results.items()}

print(
    f"Accuracy: {mean_results['Accuracy']:.2f}, F1: {mean_results['F1']:.2f}, Precision: {mean_results['Precision']:.2f}, Recall: {mean_results['Recall']:.2f}"
)

In [ ]:
colab_functions.inspect_predictions(logits=predictions, test_dataloader=test_dataloader)

# Visualize Spatial importance of features via TorchCam


In [ ]:
best_model = train_NN.Resnet_model.load_from_checkpoint(
    checkpoint_callback.best_model_path
)

In [ ]:
colab_functios.plot_activation(
    test_dataloader, device=device, model=best_model, save=False
)